# Kalman filtering

## Introduction

Much thought has been given to the interfaces of the Kalman filter and related classes in thalesians.tsa. These interfaces and the underlying implementations incorporate many suggestions by Martin Zinkin.

Before we proceed, we import some Python modules:

In [1]:
import os, sys
sys.path.append(os.path.abspath('../../main/python'))

import datetime as dt

import numpy as np
import numpy.testing as npt
import matplotlib.pyplot as plt

from thalesians.tsa.distrs import NormalDistr as N
import thalesians.tsa.filtering as filtering
import thalesians.tsa.numpyutils as npu
import thalesians.tsa.processes as proc

## A single-process, univariate example

First we need a **process model**. In this case it will be a single stochastic process,

In [2]:
process = proc.WienerProcess.createfromcov(mean=3., cov=25.)

This we pass to a newly created Kalman filter, along with the initial time and initial state. The latter takes the form of a normal distribution. We have chosen to use Python `datetime`s as our data type for time, but we could have chosen `int`s or something else.

In [3]:
t0 = dt.datetime(2017, 5, 12, 16, 18, 25, 204000)
kf = filtering.KalmanFilter(t0, statedistr=N(mean=100., cov=250.), process=process)

Next we create an **observable**, which incorporates a particular **observation model**. In this case, the observation model is particularly simple, since we are observing the entire state of the Kalman filter. Our observation model is a 1x1 identity:

In [4]:
observable = kf.createobservable(filtering.KalmanFilterObsModel.create(1.), process)

Let's roll forward the time by one hour:

In [5]:
t1 = t0 + dt.timedelta(hours=1)

What is our predicted observation at this time? Since we haven't observed any actual information, this is our **prior** observation estimate:

In [6]:
priorpredictedobs1 = observable.predict(t1)
priorpredictedobs1

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 100.125]], cov=[[ 251.04166667]]), crosscov=[[ 251.04166667]])

We confirm that this is consistent with how our (linear-Gaussian) process model scales over time:

In [7]:
priorpredictedobs1 = observable.predict(t1)
npt.assert_almost_equal(priorpredictedobs1.distr.mean, 100. + 3./24.)
npt.assert_almost_equal(priorpredictedobs1.distr.cov, 250. + 25./24.)
npt.assert_almost_equal(priorpredictedobs1.crosscov, priorpredictedobs1.distr.cov)

Let us now actually *observe* our observation. Say, the observation is 100.35 and the observation noise covariance is 100.0:

In [8]:
observable.observe(t1, N(mean=100.35, cov=100.0))

ObsResult(time=2017-05-12 17:18:25.204000, obsdistr=Normal(mean=[[ 100.35]], cov=[[ 100.]]), accepted=True, predictedobs=PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 100.125]], cov=[[ 251.04166667]]), crosscov=[[ 251.04166667]]), innovdistr=Normal(mean=[[ 0.225]], cov=[[ 351.04166667]]), loglikelihood=-3.849463)

Having seen an actual observation, let us obtain the **posterior** observation estimate:

In [9]:
posteriorpredictedobs1 = observable.predict(t1); posteriorpredictedobs1

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 100.28590504]], cov=[[ 71.51335312]]), crosscov=[[ 71.51335312]])

We can now fast-forward the time, by two hours, say, and repeat the process:

In [10]:
t2 = t1 + dt.timedelta(hours=2)
        
priorpredictedobs2 = observable.predict(t2)
npt.assert_almost_equal(priorpredictedobs2.distr.mean, 100.28590504 + 2.*3./24.)
npt.assert_almost_equal(priorpredictedobs2.distr.cov, 71.513353115 + 2.*25./24.)
npt.assert_almost_equal(priorpredictedobs2.crosscov, priorpredictedobs2.distr.cov)
        
observable.observe(t2, N(mean=100.35, cov=100.0))

posteriorpredictedobs2 = observable.predict(t2)
npt.assert_almost_equal(posteriorpredictedobs2.distr.mean, 100.45709020)
npt.assert_almost_equal(posteriorpredictedobs2.distr.cov, 42.395213845)
npt.assert_almost_equal(posteriorpredictedobs2.crosscov, posteriorpredictedobs2.distr.cov)


## A multi-process, multivariate example

The real power of our Kalman filter interface is demonstrated for process models consisting of several (independent) stochastic processes:

In [11]:
process1 = proc.WienerProcess.createfromcov(mean=3., cov=25.)
process2 = proc.WienerProcess.createfromcov(mean=[1., 4.], cov=[[36.0, -9.0], [-9.0, 25.0]])

Such models are common in finance, where, for example, the dynamics of a yield curve may be represented by a (multivariate) stochastic process, whereas the idiosyncratic spread for each bond may be an independent stochastic process.

Let us pass `process1` and `process2` as a (compound) process model to our Kalman filter, along with the initial time and state:

In [12]:
t0 = dt.datetime(2017, 5, 12, 16, 18, 25, 204000)
kf = filtering.KalmanFilter(
    t0,
    statedistr=N(
        mean=[100.0, 120.0, 130.0],
        cov=[[250.0, 0.0, 0.0],
             [0.0, 360.0, 0.0],
             [0.0, 0.0, 250.0]]),
    process=(process1, process2))

We shall now create several **observables**, each corresponding to a distinct **observation model**. The first one will observe the entire state:

In [13]:
stateobservable = kf.createobservable(
    filtering.KalmanFilterObsModel.create(1.0, np.eye(2)),
    process1, process2)

The second observable will observe the first coordinate of the first process:

In [14]:
coord0observable = kf.createobservable(
    filtering.KalmanFilterObsModel.create(1.),
    process1)

The third, the first coordinate of the second process:

In [15]:
coord1observable = kf.createobservable(
    filtering.KalmanFilterObsModel.create(npu.row(1., 0.)),
    process2)

The fourth, the second coordinate of the second process:

In [16]:
coord2observable = kf.createobservable(
    filtering.KalmanFilterObsModel.create(npu.row(0., 1.)),
    process2)

The fifth will observe the sum of the entire state (across the two processes):

In [18]:
sumobservable = kf.createobservable(
    filtering.KalmanFilterObsModel.create(npu.row(1., 1., 1.)),
    process1, process2)

And the sixth a certain linear combination thereof:

In [19]:
lincombobservable = kf.createobservable(
    filtering.KalmanFilterObsModel.create(npu.row(2., 0., -3.)),
    process1, process2)

Fast-forward the time by one hour:

In [20]:
t1 = t0 + dt.timedelta(hours=1)

Let's predict the state at this time...

In [21]:
predictedobs1_prior = stateobservable.predict(t1)
predictedobs1_prior

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 100.125     ]
 [ 120.04166667]
 [ 130.16666667]], cov=[[ 251.04166667    0.            0.        ]
 [   0.          361.5          -0.375     ]
 [   0.           -0.375       251.04166667]]), crosscov=[[ 251.04166667    0.            0.        ]
 [   0.          361.5          -0.375     ]
 [   0.           -0.375       251.04166667]])

And check that it is consistent with the scaling of the (multivariate) Wiener process with time:

In [23]:
npt.assert_almost_equal(predictedobs1_prior.distr.mean,
                        npu.col(100.0 + 3.0/24.0, 120.0 + 1.0/24.0, 130.0 + 4.0/24.0))
npt.assert_almost_equal(predictedobs1_prior.distr.cov,
                        [[250.0 + 25.0/24.0, 0.0, 0.0],
                         [0.0, 360.0 + 36.0/24.0, -9.0/24.0],
                         [0.0, -9.0/24.0, 250 + 25.0/24.0]])
npt.assert_almost_equal(predictedobs1_prior.crosscov, predictedobs1_prior.distr.cov)

Suppose that a new observation arrives, and we observe each of the three coordinates individually:

In [25]:
stateobservable.observe(t1, N(mean=[100.35, 121.0, 135.0],
                              cov=[[100.0, 0.0, 0.0],
                                   [0.0, 400.0, 0.0],
                                   [0.0, 0.0, 100.0]]));

Let's look at our (posterior) predicted state:

In [26]:
stateobservable.predict(t1)

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 100.31262976]
 [ 120.65650761]
 [ 134.19712486]], cov=[[  4.16955017e+01   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   1.28762185e+02  -2.21847868e-02]
 [  0.00000000e+00  -2.21847868e-02   4.16954948e+01]]), crosscov=[[  4.16955017e+01   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   1.28762185e+02  -2.21847868e-02]
 [  0.00000000e+00  -2.21847868e-02   4.16954948e+01]])

Let's also look at the predictions for the individual coordinates:

In [27]:
coord0observable.predict(t1)

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 100.31262976]], cov=[[ 41.69550173]]), crosscov=[[ 41.69550173   0.           0.        ]])

In [28]:
coord1observable.predict(t1)

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 120.65650761]], cov=[[ 128.76218472]]), crosscov=[[  0.00000000e+00   1.28762185e+02  -2.21847868e-02]])

In [29]:
coord2observable.predict(t1)

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 134.19712486]], cov=[[ 41.69549482]]), crosscov=[[  0.00000000e+00  -2.21847868e-02   4.16954948e+01]])

The predicted sum:

In [30]:
sumobservable.predict(t1)

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[ 355.16626223]], cov=[[ 212.1088117]]), crosscov=[[  41.69550173  128.73999994   41.67331003]])

And the predicted linear combination:

In [31]:
lincombobservable.predict(t1)

PredictedObs(time=2017-05-12 17:18:25.204000, distr=Normal(mean=[[-201.96611508]], cov=[[ 542.04146031]]), crosscov=[[  8.33910035e+01   6.65543603e-02  -1.25086484e+02]])

Let's now go 30 minutes into the future:

In [32]:
t2 = t1 + dt.timedelta(minutes=30)

And observe only the first coordinate of the second process, with a pretty high confidence:

In [34]:
coord1observable.observe(t2, N(mean=125.25, cov=4.))

ObsResult(time=2017-05-12 17:48:25.204000, obsdistr=Normal(mean=[[ 125.25]], cov=[[ 4.]]), accepted=True, predictedobs=PredictedObs(time=2017-05-12 17:48:25.204000, distr=Normal(mean=[[ 120.67734094]], cov=[[ 129.51218472]]), crosscov=[[   0.          129.51218472   -0.20968479]]), innovdistr=Normal(mean=[[ 4.57265906]], cov=[[ 133.51218472]]), loglikelihood=-3.444339)

How does our predicted state change?

In [36]:
stateobservable.predict(t2)

PredictedObs(time=2017-05-12 17:48:25.204000, distr=Normal(mean=[[ 100.37512976]
 [ 125.11300399]
 [ 134.2732767 ]], cov=[[  4.22163351e+01   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   3.88016075e+00  -6.28211686e-03]
 [  0.00000000e+00  -6.28211686e-03   4.22159988e+01]]), crosscov=[[  4.22163351e+01   0.00000000e+00   0.00000000e+00]
 [  0.00000000e+00   3.88016075e+00  -6.28211686e-03]
 [  0.00000000e+00  -6.28211686e-03   4.22159988e+01]])

Thirty minutes later...

In [37]:
t3 = t2 + dt.timedelta(minutes=30)

We observe the *sum* of the three coordinates, rather than the individual coordinates:

In [39]:
sumobservable.observe(t3, N(mean=365.00, cov=9.))

ObsResult(time=2017-05-12 18:18:25.204000, obsdistr=Normal(mean=[[ 365.]], cov=[[ 9.]]), accepted=True, predictedobs=PredictedObs(time=2017-05-12 18:18:25.204000, distr=Normal(mean=[[ 359.92807712]], cov=[[ 89.71659709]]), crosscov=[[ 42.7371684    4.43637863  42.54305006]]), innovdistr=Normal(mean=[[ 5.07192288]], cov=[[ 98.71659709]]), loglikelihood=-3.345359)

How has our prediction of the state changed?

In [40]:
stateobservable.predict(t3)

PredictedObs(time=2017-05-12 18:18:25.204000, distr=Normal(mean=[[ 102.63340664]
 [ 125.36177235]
 [ 136.54241339]], cov=[[ 24.23505612  -1.92063206 -18.41807303]
 [ -1.92063206   4.43078743  -2.10569039]
 [-18.41807303  -2.10569039  24.40241667]]), crosscov=[[ 24.23505612  -1.92063206 -18.41807303]
 [ -1.92063206   4.43078743  -2.10569039]
 [-18.41807303  -2.10569039  24.40241667]])

And what is its predicted sum?

In [41]:
sumobservable.predict(t3)

PredictedObs(time=2017-05-12 18:18:25.204000, distr=Normal(mean=[[ 364.53759239]], cov=[[ 8.17946928]]), crosscov=[[ 3.89635104  0.40446499  3.87865325]])